In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/transactions_cleaned.csv')
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Publication Date'] = pd.to_datetime(df['Publication Date'])
df

,Politician Name,Party,Chamber,State,Issuer Name,Ticker,Publication Date,Transaction Date,Reporting Gap,Owner,Transaction Type,Value Range,Price,Transaction ID,Price_0_days,Approx Transaction Amount,Approx Share Count
0,Tom Carper,Democrat,Senate,DE,Valero Energy Corp,VLO,2024-05-01,2024-04-11,19,Spouse,SELL,1K–15K,177.04,10000062280,177.039993,8000.0,45.0
1,Tom Carper,Democrat,Senate,DE,Enbridge Inc,ENB,2024-05-01,2024-04-11,19,Spouse,BUY,1K–15K,34.55,10000062281,34.549999,8000.0,231.0
2,Dan Newhouse,Republican,House,WA,Accenture PLC,ACN,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,324.40,20003772725,324.399994,8000.0,24.0
3,Dan Newhouse,Republican,House,WA,Analog Devices Inc,ADI,2024-04-26,2024-04-10,15,Spouse,BUY,1K–15K,196.33,20003772726,196.330002,8000.0,40.0
4,Dan Newhouse,Republican,House,WA,Booking Holdings Inc,BKNG,2024-04-26,2024-04-10,15,Spouse,SELL,1K–15K,3586.13,20003772728,3586.129883,8000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31584,Alan Lowenthal,Democrat,House,CA,Americold Realty Trust,COLD,2021-03-24,2021-03-15,8,Spouse,SELL,15K–50K,37.13,20001621840,37.130001,32500.0,875.0
31585,Mark Green,Republican,House,TN,NGL Energy Partners LP,NGL,2021-03-21,2021-03-17,3,Undisclosed,BUY,15K–50K,2.34,20001620394,2.340000,32500.0,13888.0
31586,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Spouse,SELL,15K–50K,70.16,20001621844,70.160004,32500.0,463.0
31587,Doug Lamborn,Republican,House,CO,NetApp Inc,NTAP,2021-03-20,2021-03-19,0,Undisclosed,SELL,1K–15K,70.16,20001621847,70.160004,8000.0,114.0


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import panel as pn
import panel.widgets as pnw

pn.extension("plotly", "tabulator", theme='dark')
pn.extension()

In [ ]:
party_colors = {
        'Democrat': '#0015BC',
        'Republican': '#DE0100',
        'Other': '#777777',
    }

In [155]:
group_by_selector = pnw.Select(options=['Politician Name', 'Party', 'Chamber', 'State', 'Issuer Name'])

plot_type_selector = pn.widgets.RadioButtonGroup(options=["Bar Plot", "Pie Chart"])
plot_functions = {"Bar Plot": px.bar, "Pie Chart": px.pie}

show_entries_count = pnw.EditableIntSlider(start=10, end=50, step=1, value=10)


def update_plot(event):
    col = group_by_selector.value
    plot_func = plot_functions[plot_type_selector.value]
    data = df.groupby(col).size().reset_index(name='Transaction Count').sort_values('Transaction Count').tail(show_entries_count.value)

    if plot_type_selector.value == 'Pie Chart':
        if col == 'Politician Name':
            data = data.merge(df[['Politician Name', 'Party', 'Chamber', 'State']].drop_duplicates(), on='Politician Name')
            fig = plot_func(data, names='Politician Name', values='Transaction Count', color='Party', color_discrete_map=party_colors)

        elif col == 'Party':
            fig = plot_func(data, names='Party', values='Transaction Count', color='Party', color_discrete_map=party_colors)
        else:
            fig = plot_func(data, names=col, values='Transaction Count')

    else:
        if col == 'Politician Name':
            data = data.merge(df[['Politician Name', 'Party', 'Chamber', 'State']].drop_duplicates(), on='Politician Name')
            fig = plot_func(data, x='Transaction Count', y='Politician Name', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        elif col == 'Party':
            fig = plot_func(data, x='Transaction Count', y='Party', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        else:
            fig = plot_func(data, x=data['Transaction Count'], y=data.columns[0])

        fig.update_yaxes(categoryorder='total ascending')

    fig.update_layout(
        title={
            'text': "Distribution of Transaction Count by " + col,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )

    plot_pane.object = fig


group_by_selector.param.watch(update_plot, 'value')
plot_type_selector.param.watch(update_plot, 'value')
show_entries_count.param.watch(update_plot, 'value')

plot_pane = pn.pane.Plotly(sizing_mode='stretch_width', height=600)

update_plot(None)

transaction_count_analysis = pn.Column(pn.Row("# Transaction Count Analysis"),
                                      pn.Row("Group by", group_by_selector),
                                      pn.Row("Show Top", show_entries_count),
                                      pn.Row("Plot Type", plot_type_selector),
                                      plot_pane
                                      ,name= 'Transaction Count Analysis'
                                      )
# TODO: Fix Pie Chart Hover Data

In [156]:
tabs = pn.Tabs(transaction_count_analysis
            #    trades_analysis,
            #    performance_analysis
               )

full_dashboard = pn.Row(pn.Column("# Lets Investigate Politician Stock Trading Together!",
                           "## Select the Area",
                           tabs,
                        #    recruiting,
                        #    linked_dataset
                                  ))
full_dashboard.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Tabs
            [0] Column(name='Transaction C...)
                [0] Row
                    [0] Markdown(str)
                [1] Row
                    [0] Markdown(str)
                    [1] Select(options=['Politician Name', ...], value='Politician Name')
                [2] Row
                    [0] Markdown(str)
                    [1] EditableIntSlider(end=50, start=10, value=10, width=300)
                [3] Row
                    [0] Markdown(str)
                    [1] RadioButtonGroup(options=['Bar Plot', 'Pie Chart'], value='Bar Plot')
                [4] Plotly(Figure, height=600, sizing_mode='stretch_width')